In [ ]:
from lib.imports import *

In [ ]:
skip_idx = [19,24,26,32,34,38,40,45,52,55,70]
train_idx = [0,1,2,3,4,5]
window_size = 101
X,y = load_and_window_nursing_list_for_convolution(train_idx,window_size=window_size)
X_train,X_dev,y_train,y_dev = train_test_split(X,y,test_size=.5,stratify=y)
trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=32,shuffle=True)
devloader = DataLoader(TensorDataset(X_dev,y_dev),batch_size=32,shuffle=True)

In [ ]:
from torch import nn
class FCN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.c1 = nn.Conv1d(in_channels=3,out_channels=128,kernel_size=8,stride=1)
        self.bn1 = nn.BatchNorm1d(num_features=128)

        self.c2 = nn.Conv1d(in_channels=128,out_channels=256,kernel_size=5,stride=1)
        self.bn2 = nn.BatchNorm1d(num_features=256)

        self.c3 = nn.Conv1d(in_channels=256,out_channels=128,kernel_size=3,stride=1)
        self.bn3 = nn.BatchNorm1d(num_features=128)

        self.gap = nn.AvgPool1d(kernel_size=88)
        self.fc1 = nn.Linear(in_features=128,out_features=1)
    def forward(self,x):
        x = self.c1(x)
        x = self.bn1(x)
        x = relu(x)

        x = self.c2(x)
        x = self.bn2(x)
        x = relu(x)

        x = self.c3(x)
        x = self.bn3(x)
        x = relu(x)

        x = self.gap(x)
        x = self.fc1(x.squeeze(2))
        return x
device = 'cuda'
model = FCN().to(device=device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())
params = sum([p.flatten().size()[0] for p in list(model.parameters())])
print("Params: ",params)

In [ ]:
loss_tr = []
loss_dev = []
pbar = tqdm(range(20))

for epoch in pbar:
    # train loop
    model.train()
    loss_tr_total = 0
    for (X_tr,y_tr) in trainloader:
        X_tr,y_tr = X_tr.to(device),y_tr.to(device)
        logits = model(X_tr)
        loss = criterion(logits,y_tr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_tr_total += loss.item()
    loss_tr.append(loss_tr_total/len(trainloader))

    # dev loop
    model.eval()
    loss_dev_total = 0
    for (X_dv,y_dv) in devloader:
        X_dv,y_dv = X_dv.to(device),y_dv.to(device)
        logits = model(X_dv)
        loss = criterion(logits,y_dv)
        loss_dev_total += loss.item()
    loss_dev.append(loss_dev_total/len(devloader))
plt.plot(loss_tr,label='train')
plt.plot(loss_dev,label='dev')
plt.legend()

In [ ]:
test_evaluation(devloader,model,criterion)

In [ ]:
from sklearn.metrics import f1_score
f1i = []
for idx in tqdm(range(71)):
    if(idx in skip_idx or idx in train_idx):
        continue
    X,y = load_and_window_nursing_list_for_convolution([idx])
    loss,y_true,y_pred = test_evaluation(DataLoader(TensorDataset(X,y),batch_size=32,shuffle=True),model,criterion,plot=True)
    f1i.append(f1_score(y_true=y,y_pred=y_pred.round(),average='macro'))

In [ ]:
sns.kdeplot(f1i,bw_adjust=.4)
print(torch.tensor(f1i).mean()) #.4958
print(torch.tensor(f1i).std()) #.0063